# 工具包导入&数据读取
## 工具包导入

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
import gc
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline 

In [46]:
%pwd

u'/home/t-zhga/Private/tianchi/scripts'

## 数据读取
- 为了方便分析，我们读取3000万条数据进行处理

In [52]:
# path = '../final_input/'
# train = pd.read_csv(path + 'final_train.csv',nrows=1000000)
# test = pd.read_csv(path + 'final_test.csv',nrows=1000000)

In [112]:
path = '../final_input/'
train = pd.read_csv(path + 'final_train.csv')
#test = pd.read_csv(path + 'final_test.csv')

# 特征工程 & 验证结果(1-Gram)

In [113]:
train_data = train[['file_id','label']].drop_duplicates()
train_data.head()

,file_id,label
0,1,5
6786,2,2
7602,3,0
8065,4,0
10111,5,0


In [114]:
train_data['label'].value_counts()

0    4978
5    4289
7    1487
2    1196
3     820
6     515
1     502
4     100
Name: label, dtype: int64

## 全局特征:
- File_id (Api): count,nunique
- File_id (Tid): count,nunique,max,min,quantile(20,40,50,60,80),std,range
- File_id (Return Value): count,nunique,max,min,quantile(20,40,50,60,80),std,range
- File_id (Index): count,nunique,max,min,quantile(20,40,50,60,80),std,range

### File_id (Api): count,nunique

In [115]:
train.head()

,file_id,label,api,tid,index
0,1,5,LdrLoadDll,2488,0
1,1,5,LdrGetProcedureAddress,2488,1
2,1,5,LdrGetProcedureAddress,2488,2
3,1,5,LdrGetProcedureAddress,2488,3
4,1,5,LdrGetProcedureAddress,2488,4


In [116]:
api_opt = ['count','nunique'] 
for opt in api_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['api'].agg({'fileid_api_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id')  

count


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


In [117]:
train_data.head()

,file_id,label,fileid_api_count,fileid_api_nunique
0,1,5,6786,116
1,2,2,816,30
2,3,0,463,42
3,4,0,2046,51
4,5,0,10002,65


### File_id (Tid): count,nunique,max,min,quantile(20,40,50,60,80),std,range

In [118]:
tid_opt = ['count','nunique','max','min','median','std'] 
for opt in tid_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['tid'].agg({'fileid_tid_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id') 

count


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique
max
min
median
std


In [ ]:
secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_tid_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['tid'].quantile(sec).values
 
train_data['fileid_tid_range'] = train.groupby(['file_id'])['tid'].quantile(0.975).values - train.groupby(['file_id'])['tid'].quantile(0.0125).values

### File_id (Index): count,nunique,max,min,quantile(20,40,50,60,80),std,range

In [ ]:
index_opt = ['count','nunique','max','min','median','std'] 
for opt in index_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['index'].agg({'fileid_index_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id') 

count


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


In [ ]:
secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_index_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['index'].quantile(sec).values
 
train_data['fileid_index_range'] = train.groupby(['file_id'])['index'].quantile(0.975).values - train.groupby(['file_id'])['index'].quantile(0.0125).values

### 全局特征的线下验证 <font color=red>( 0.0969482)</font>

#### 评估指标

In [ ]:
def lgb_logloss(preds,data):
    labels_ = data.get_label()
    classes_ = np.unique(labels_) 
    preds_prob = []
    for i in range(len(classes_)):
        preds_prob.append(preds[i*len(labels_):(i+1) * len(labels_)])
    preds_prob_ = np.vstack(preds_prob) 
    
    loss = [] 
    for i in range(preds_prob_.shape[1]):  # 样本个数
        sum_ = 0  
        for j in range(preds_prob_.shape[0]): #类别个数
            pred = preds_prob_[j,i] # 第i个样本预测为第j类的概率
            if  j == labels_[i]:
                sum_ += np.log(pred)
            else:
                sum_ += np.log(1 - pred) 
             
        loss.append(sum_)  
         
    return 'loss is: ' ,-1 * (np.sum(loss) / preds_prob_.shape[1]),False

#### 训练特征 & 标签

In [ ]:
train_features = [col for col in train_data.columns if col!='label' and col!='file_id']
train_label = 'label'

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split( train_data[train_features],train_data[train_label].values, test_size = 0.33) 
del _
gc.collect()

train_ind = train_X.index
test_ind = test_X.index

In [ ]:
# dtrain = lgb.Dataset(train_X,train_Y) 
# dval   = lgb.Dataset(test_X,test_Y, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':8,
#         #'min_data_in_leaf': 40,
#         'min_data_in_leaf': 1,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 10,
#         'random_state':100
#     }  
# lgb_model_0_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### 全局特征扩充
- File_id + return_value分段：计数

## 局部组合特征(展开形式)
### File_id + Api  
- File_id + Api (tid): count,nunique
- File_id + Api (return value): nunique, max, min, median, std
- File_id + Api (index):  nunique, max, min, median, std



#### File_id + Api (tid): count,nunique

In [ ]:
def groupby_pivot_features(data_merge, data_orig , groupby_features,col1 = None, col2 = None, opts = None):
    for opt in opts:
        print(opt)
        train_split = data_orig.groupby(['file_id',col1])[col2].agg({'fileid_' + col1 + '_'+col2+'_'+ str(opt):opt}).reset_index() 
        
        train_split_ =  pd.pivot_table(train_split, values = 'fileid_' + col1 + '_'+col2+'_'+ str(opt), index=['file_id'],columns=[col1])
        new_cols = [ 'fileid_' + col1 + '_'+col2+  '_' + opt + '_' + str(col) for col in train_split_.columns]
        
        groupby_features.append(new_cols)
        train_split_.columns = new_cols 

        train_split_.reset_index(inplace = True)
        
        data_merge = pd.merge(data_merge,train_split_,how='left', on='file_id') 
    return data_merge,groupby_features 
    

In [ ]:
groupby_features = []
api_opts = ['count', 'nunique']
train_data_,groupby_features = groupby_pivot_features(train_data, train, groupby_features, col1 = 'api', col2 = 'tid', opts = api_opts)

#### File_id + Api (return value): nunique, max, min, median, std

In [ ]:
# api_opts = ['nunique','max','min','median','std']
# train_data_,groupby_features = groupby_pivot_features(train_data_, train, groupby_features, col1 = 'api', col2 = 'return_value', opts = api_opts) 

####  File_id + Api(index): nunique, max, min, median, std

In [ ]:
api_opts = ['nunique','max','min','median','std']
train_data_,groupby_features = groupby_pivot_features(train_data_, train, groupby_features, col1 = 'api', col2 = 'index', opts = api_opts) 

In [ ]:
train_data_.head()

### 1阶特征的线下验证(File_id + Api)（<font color=red>0.0347293</font>）

### File_id + Index  
- File_id + Index (api): count,nunique
- File_id + Index (return value): nunique, max, min, median, std(暂时先搁置)
- File_id + Index (tid):  nunique, max, min, median, std(暂时先搁置)


#### File_id +Tid (api): count,nunique

#### File_id + Index特征过拟合，删除


In [ ]:
# delcol = []
# for i in range(2):
#     for item in groupby_features2[i]:
#         delcol.append(item)

In [ ]:
# train_data_.drop(delcol,axis=1,inplace=True)

## 特征补充（加入index的差值特征）
- File_id + Api (index_diff): 'nunique','max','min','median','std'

In [ ]:
train_diff = train.groupby(['file_id','tid'])['index'].diff().fillna(-999).values

In [ ]:
train['index_diff'] = train_diff

In [ ]:
train_diff = train.loc[train.index_diff!=-999] 

In [ ]:
api_opts = ['nunique','max','min','median','std']
train_data_,groupby_features = groupby_pivot_features(train_data_, train_diff, groupby_features, col1 = 'api', col2 = 'index_diff', opts = api_opts) 

### 线下验证(<font color=red>0.0346954</font>)

In [ ]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### 删除quantile,std统计变量之后的验证(<font color=red>0.0350054</font>)  

In [ ]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id' and 'std' not in col and 'quantile' not in col]
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [ ]:
# train_data_.to_csv('/data/Data_JieZhang/TC_SAFE/train_val/train_data.csv',index = None) 

# 特征工程& 验证结果 2-Gram
## 全局特征
### File_id（Api_2）:count,nunique

In [ ]:
train['api_shift'] = train['api'].shift(-1)
train['api_2'] = train['api'] +'_' + train['api_shift']

In [ ]:
train.drop(['api_shift'],axis=1,inplace=True)

In [ ]:
api_count = train['api_2'].value_counts()

In [ ]:
api_opt = ['count','nunique'] 
for opt in api_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['api_2'].agg({'fileid_api_2_' + opt: opt}).reset_index() 
    train_data_ = pd.merge(train_data_,tmp,how='left', on='file_id')  

## 局部特征
### File_id + tid (Api_2): count特征

In [ ]:
api_value_counts = pd.DataFrame(api_count).reset_index()
api_value_counts.columns = ['api_2','api_2_count']

In [ ]:
train = pd.merge(train, api_value_counts, on ='api_2' , how='left')

In [ ]:
api_opts = ['count']
groupby_features =  []
train_data_,groupby_features = groupby_pivot_features(train_data_, train.loc[train.api_2_count>=20], groupby_features, col1 = 'api_2', col2 = 'tid', opts = api_opts)

### 线下验证(<font color=red> 0.0330886</font>)

In [ ]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### File_id + index (Api_2): max,min特征

In [ ]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
train_label = 'label'
print(len(train_features))
train_X, test_X, train_Y, test_Y = train_test_split( train_data_[train_features],train_data_[train_label].values, test_size = 0.33) 

dtrain = lgb.Dataset(train_X,train_Y) 
dval   = lgb.Dataset(test_X,test_Y, reference = dtrain) 

params = {
        'task':'train', 
        'num_leaves': 255,
        'objective': 'multiclass',
        'num_class':8,
        #'min_data_in_leaf': 40,
        'min_data_in_leaf': 10,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
        'bagging_freq': 5, 
        'max_bin':128,
        'num_threads': 64,
        'random_state':100
    }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [ ]:
fea_imp = pd.DataFrame({'feature':train_features, 'imp':lgb_model_3_order.feature_importance()}).sort_values('imp')
important_features = fea_imp.loc[fea_imp.imp >=1, 'feature'].values
important_features = list(important_features)

important_features.append('file_id')
important_features.append('label')

In [ ]:
# train_data_[important_features].to_csv('../feature_final/train_data_2gram.csv',index = None)

train_ind = train_X.index
test_ind = test_X.index

In [ ]:
# len(important_features)

In [ ]:
api_opts = ['max','min']
train_data_,groupby_features = groupby_pivot_features(train_data_[important_features], train.loc[train.api_2_count>=20], groupby_features, col1 = 'api_2', col2 = 'index', opts = api_opts)

In [ ]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id' and 'std' not in col and 'quantile' not in col]
train_label = 'label'

train_ind = train_X.index
test_ind = test_X.index

dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

params = {
        'task':'train', 
        'num_leaves': 255,
        'objective': 'multiclass',
        'num_class':8,
        'min_data_in_leaf': 10,
        #'min_data_in_leaf': 1,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
        'bagging_freq': 5, 
        'max_bin':128,
        'num_threads': 64,
        'random_state':100
    }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [ ]:
fea_imp = pd.DataFrame({'feature':train_features, 'imp':lgb_model_3_order.feature_importance()}).sort_values('imp')
important_features = fea_imp.loc[fea_imp.imp >=1, 'feature'].values
important_features = list(important_features)

important_features.append('file_id')
important_features.append('label')

train_data_[important_features].to_csv('../feature_final/train_data_2gram.csv',index = None)
 

# 附录
tf-idf的1Gram特征可以替换api的次数特征等，加入tf-idf有提升，提升较小

In [4]:
# import pandas as pd
# d = pd.read_csv('input/test_data_2gram.csv')
# d.shape